In [19]:
import os
import json
import torch
import torchvision
import torch.nn as nn
import numpy as np
import cv2
import gradio as gr
from skimage.feature import graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier
import joblib

d:\MED_LEAF_ID-1\base\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
json_path = r"D:\MED_LEAF_ID\class_labels.json"

In [20]:
if not os.path.exists(json_path):
    print("⚠️ class_labels.json not found! Generating...")
    dataset_path = r"D:\MED_LEAF_ID-1\dataset\Medicinal Leaf dataset"
    class_labels = {str(idx): class_name for idx, class_name in enumerate(sorted(os.listdir(dataset_path)))}
    with open(json_path, "w") as f:
        json.dump(class_labels, f, indent=4)
    print(f"✅ Class labels saved to {json_path}")
else:
    with open(json_path, "r") as f:
        class_labels = json.load(f)

⚠️ class_labels.json not found! Generating...
✅ Class labels saved to D:\MED_LEAF_ID\class_labels.json


In [21]:
def get_class_name(pred_class):
    return class_labels.get(str(pred_class), "Unknown Class")


In [22]:
class EfficientNetB0Classifier(nn.Module):
    def __init__(self, num_classes):
        super(EfficientNetB0Classifier, self).__init__()
        self.base_model = torchvision.models.efficientnet_b0(pretrained=True)
        self.base_model.classifier[1] = nn.Linear(self.base_model.classifier[1].in_features, num_classes)
        self.base_model.classifier.add_module('dropout', nn.Dropout(0.2))

    def forward(self, x):
        return self.base_model(x)


In [24]:
cnn_model_path = r"D:\MED_LEAF_ID-1\models\efficientnetb0_leaf_model.pth"
cnn_model = EfficientNetB0Classifier(num_classes=80)
cnn_model.load_state_dict(torch.load(cnn_model_path, map_location="cpu"))
cnn_model.eval()

C:\Users\aadhi\AppData\Local\Temp\ipykernel_30320\1827284094.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cnn_model.load_state_dict(torch.load(cnn_model_path, map_loc

EfficientNetB0Classifier(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
      

In [29]:
rf_model_path = r"D:\MED_LEAF_ID-1\models\plant_classifier.pkl"
rf_model = joblib.load(rf_model_path)

In [60]:
import cv2
import numpy as np
from skimage.feature import graycomatrix, graycoprops

def extract_glcm_features(image):
    try:
        if image is None:
            raise ValueError("⚠️ Error: Image is None. Check if it's loaded correctly.")

        print(f"📸 Image Shape Before Processing: {image.shape}")  # Debugging

        # Convert to grayscale
        if len(image.shape) == 3:  # RGB/BGR Image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:  # Already grayscale
            gray = image  

        print(f"🔹 Grayscale Image Shape: {gray.shape}")  # Debugging

        # Check pixel value range
        if gray.min() < 0 or gray.max() > 255:
            print(f"⚠️ Warning: Image pixel values out of range (Min: {gray.min()}, Max: {gray.max()})")
            gray = np.clip(gray, 0, 255).astype(np.uint8)

        # Compute GLCM
        distances = [1, 2]  # Adjust this based on training setup
        angles = [0, np.pi/4, np.pi/2]  # Adjust this based on training setup

        glcm = graycomatrix(gray, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)

        # Extract features
        features = np.hstack([
        graycoprops(glcm, 'contrast').flatten(),
        graycoprops(glcm, 'dissimilarity').flatten(),
        graycoprops(glcm, 'homogeneity').flatten(),
        graycoprops(glcm, 'energy').flatten(),
        graycoprops(glcm, 'correlation').flatten(),
    # graycoprops(glcm, 'ASM').flatten(),  # Comment this
        ])



        print(f"✅ Extracted GLCM Features: {len(features)}")  # Debugging

        # Ensure the correct number of features
        if len(features) != 30:
            raise ValueError(f"⚠️ Error: Extracted {len(features)} features instead of 30.")

        return features
    except Exception as e:
        print(f"⚠️ GLCM feature extraction failed! Error: {str(e)}")
        return None


In [61]:
def predict(image):
    try:
        # Convert RGB (Gradio) to BGR (OpenCV)
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # CNN Prediction
        img_resized = cv2.resize(image, (224, 224)) / 255.0
        img_tensor = torch.tensor(img_resized, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0)
        cnn_pred = torch.argmax(cnn_model(img_tensor)).item()

        # GLCM + RandomForest Prediction
        glcm_features = extract_glcm_features(image)
        if glcm_features is None or len(glcm_features) != 30:
            return "⚠️ Error: GLCM feature extraction failed!"

        rf_pred = rf_model.predict([glcm_features])[0]

        # Weighted Ensemble (70% CNN, 30% RF)
        final_pred = cnn_pred if np.random.rand() < 0.7 else rf_pred
        plant_name = get_class_name(final_pred)

        return f"🌿 Predicted Plant: {plant_name}"
    except Exception as e:
        return f"⚠️ Error in prediction: {str(e)}"

# Load class labels for debugging
with open(json_path, "r") as f:
    class_mapping = json.load(f)


In [62]:
import json

# Load class labels
json_path = r"D:\MED_LEAF_ID\class_labels.json"
with open(json_path, "r") as f:
    class_mapping = json.load(f)

# Example: Your model predicts class index 0
predicted_class_index = 0  # Replace this with your actual model prediction

# Get the plant name
plant_name = class_mapping.get(str(predicted_class_index), "Unknown")

print(f"🌿 Predicted Plant Name: {plant_name}")


🌿 Predicted Plant Name: Aloevera


In [63]:
iface = gr.Interface(fn=predict, inputs=gr.Image(type="numpy"), outputs="text")
iface.launch()

* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.


📸 Image Shape Before Processing: (600, 450, 3)
🔹 Grayscale Image Shape: (600, 450)
✅ Extracted GLCM Features: 30


d:\MED_LEAF_ID-1\base\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


📸 Image Shape Before Processing: (512, 1024, 3)
🔹 Grayscale Image Shape: (512, 1024)
✅ Extracted GLCM Features: 30


d:\MED_LEAF_ID-1\base\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


...

d:\MED_LEAF_ID-1\base\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


📸 Image Shape Before Processing: (224, 224, 3)
🔹 Grayscale Image Shape: (224, 224)
✅ Extracted GLCM Features: 30


d:\MED_LEAF_ID-1\base\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


📸 Image Shape Before Processing: (224, 224, 3)
🔹 Grayscale Image Shape: (224, 224)
✅ Extracted GLCM Features: 30


d:\MED_LEAF_ID-1\base\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
